In [40]:
#Import Libraries
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

In [65]:
data = pd.read_csv(r"C:\Users\Lenovo\Documents\GitHub\ProgrammingForDataScienceProject\Data\nation.1751_2014.csv")
print(data)

            Nation  Year  \
0      AFGHANISTAN  1949   
1      AFGHANISTAN  1950   
2      AFGHANISTAN  1951   
3      AFGHANISTAN  1952   
4      AFGHANISTAN  1953   
...            ...   ...   
17227     ZIMBABWE  2010   
17228     ZIMBABWE  2011   
17229     ZIMBABWE  2012   
17230     ZIMBABWE  2013   
17231     ZIMBABWE  2014   

       Total CO2 emissions from fossil-fuels and cement production (thousand metric tons of C)  \
0                                                      4                                         
1                                                     23                                         
2                                                     25                                         
3                                                     25                                         
4                                                     29                                         
...                                                  ...                   

In [66]:
data.head(10)

,Nation,Year,Total CO2 emissions from fossil-fuels and cement production (thousand metric tons of C),Emissions from solid fuel consumption,Emissions from liquid fuel consumption,Emissions from gas fuel consumption,Emissions from cement production,Emissions from gas flaring,Per capita CO2 emissions (metric tons of carbon),Emissions from bunker fuels (not included in the totals)
0,AFGHANISTAN,1949,4,4,0,0,0,NaN,NaN,0
1,AFGHANISTAN,1950,23,6,18,0,0,0,0,0
2,AFGHANISTAN,1951,25,7,18,0,0,0,0,0
3,AFGHANISTAN,1952,25,9,17,0,0,0,0,0
4,AFGHANISTAN,1953,29,10,18,0,0,0,0,0
5,AFGHANISTAN,1954,29,12,18,0,0,0,0,0
6,AFGHANISTAN,1955,42,17,25,0,0,0,0,0
7,AFGHANISTAN,1956,50,17,33,0,0,0,0.01,0
8,AFGHANISTAN,1957,80,21,59,0,0,0,0.01,0
9,AFGHANISTAN,1958,90,25,65,0,0,0,0.01,0


In [67]:
#return size of rows and cols
data.shape

(17232, 10)

In [68]:
data.index

RangeIndex(start=0, stop=17232, step=1)

In [69]:
data.columns

Index(['Nation', 'Year',
       'Total CO2 emissions from fossil-fuels and cement production (thousand metric tons of C)',
       'Emissions from solid fuel consumption',
       'Emissions from liquid fuel consumption',
       'Emissions from gas fuel consumption',
       'Emissions from cement production', 'Emissions from gas flaring',
       'Per capita CO2 emissions (metric tons of carbon)',
       'Emissions from bunker fuels (not included in the totals)'],
      dtype='object')

In [70]:
data.loc[0]

Nation                                                                                     AFGHANISTAN
Year                                                                                              1949
Total CO2 emissions from fossil-fuels and cement production (thousand metric tons of C)              4
Emissions from solid fuel consumption                                                                4
Emissions from liquid fuel consumption                                                               0
Emissions from gas fuel consumption                                                                  0
Emissions from cement production                                                                     0
Emissions from gas flaring                                                                         NaN
Per capita CO2 emissions (metric tons of carbon)                                                   NaN
Emissions from bunker fuels (not included in the totals)                 

In [71]:
data.dtypes

Nation                                                                                     object
Year                                                                                        int64
Total CO2 emissions from fossil-fuels and cement production (thousand metric tons of C)     int64
Emissions from solid fuel consumption                                                      object
Emissions from liquid fuel consumption                                                     object
Emissions from gas fuel consumption                                                        object
Emissions from cement production                                                            int64
Emissions from gas flaring                                                                 object
Per capita CO2 emissions (metric tons of carbon)                                           object
Emissions from bunker fuels (not included in the totals)                                    int64
dtype: object

In [57]:
#find missing values
data.isna().sum()

Nation                                                                                      0
Year                                                                                        0
Total CO2 emissions from fossil-fuels and cement production (thousand metric tons of C)     0
Emissions from solid fuel consumption                                                       0
Emissions from liquid fuel consumption                                                      0
Emissions from gas fuel consumption                                                         0
Emissions from cement production                                                            0
Emissions from gas flaring                                                                 58
Per capita CO2 emissions (metric tons of carbon)                                           58
Emissions from bunker fuels (not included in the totals)                                    0
dtype: int64

There are missing values in national dataset.

In [62]:
#data is numeric, fill missing values with mean
data['Per capita CO2 emissions (metric tons of carbon)'].fillna(data['Per capita CO2 emissions (metric tons of carbon)'].mean(),inplace=True)

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [ ]:
data.notnull()